In [141]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np


In [142]:
df = load_breast_cancer()
X = df.data
y = df.target

In [143]:
#스케일
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [144]:
#스플릿
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [145]:
X_train_t = torch.tensor(X_train, dtype=torch.float32)
X_test_t  = torch.tensor(X_test, dtype=torch.float32)

# 타겟값 이진분류니 형태변경,분리를 잘못넣어서 오류
y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_t  = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


#넘파이 배열 > 텐서 배열로 변환
train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)

In [146]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [147]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [148]:
class Cancerclf(nn.Module):
    def __init__(self):
        super(Cancerclf,self).__init__()
        self.relu = nn.ReLU()
        self.lr1 = nn.Linear(30,64) #입력 30개인데 32라써서 오류
        self.lr2 = nn.Linear(64,32) #히든
        self.lr3 = nn.Linear(32,1) #히든받고 출력

    def forward(self,x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        x = self.relu(x)
        x = self.lr3(x)
        return x #대문자 x써서 오류


In [149]:
model = Cancerclf().to(device)


wtf = nn.BCEWithLogitsLoss()  # 손실 함수
opti = optim.Adam(model.parameters(), lr =0.001)

In [150]:
import torch.nn as nn


def train_model(epochs):
    model.train()
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            opti.zero_grad()
            outputs = model(inputs)

            # outputs: (N,1), labels: (N,1) 조건 필요
            loss = wtf(outputs, labels)
            loss.backward()
            opti.step()

        print(f"Epoch [{epoch+1}/{epochs}] Loss: {loss.item():.4f}")


In [151]:
def evaluate_model():
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs,labels in test_loader:
            inputs,labels = inputs.to(device),labels.to(device)
            outputs = model(inputs)

            predicted = torch.round(torch.sigmoid(outputs))
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accuarcy = 100 * correct / total
        print(f"T정확도:{accuarcy:.2f}%")

In [152]:
train_model(epochs=100)
evaluate_model()

Epoch [1/100] Loss: 0.5944
Epoch [2/100] Loss: 0.2584
Epoch [3/100] Loss: 0.1681
Epoch [4/100] Loss: 0.1276
Epoch [5/100] Loss: 0.0855
Epoch [6/100] Loss: 0.1432
Epoch [7/100] Loss: 0.1879
Epoch [8/100] Loss: 0.0115
Epoch [9/100] Loss: 0.0092
Epoch [10/100] Loss: 0.0363
Epoch [11/100] Loss: 0.0109
Epoch [12/100] Loss: 0.0393
Epoch [13/100] Loss: 0.0294
Epoch [14/100] Loss: 0.0068
Epoch [15/100] Loss: 0.0186
Epoch [16/100] Loss: 0.0397
Epoch [17/100] Loss: 0.4122
Epoch [18/100] Loss: 0.0130
Epoch [19/100] Loss: 0.0439
Epoch [20/100] Loss: 0.0019
Epoch [21/100] Loss: 0.0006
Epoch [22/100] Loss: 0.0242
Epoch [23/100] Loss: 0.0204
Epoch [24/100] Loss: 0.0015
Epoch [25/100] Loss: 0.2990
Epoch [26/100] Loss: 0.0016
Epoch [27/100] Loss: 0.0004
Epoch [28/100] Loss: 0.0391
Epoch [29/100] Loss: 0.0120
Epoch [30/100] Loss: 0.0053
Epoch [31/100] Loss: 0.0002
Epoch [32/100] Loss: 0.0027
Epoch [33/100] Loss: 0.0040
Epoch [34/100] Loss: 0.0013
Epoch [35/100] Loss: 0.0007
Epoch [36/100] Loss: 0.0022
E

---

# 윤년

In [160]:
def isLeap(year): 
    if year%4 == 0 and year%100!=0 or year%400==0: # %는 나머지 연산자. and로 두가지중 하나가 만족하거나 다른 1가지가 만족하면 윤년. 값이 4년이거나 100년이아닐때, 400년이 지날때
        return 1
    return 0

1~4000까지의 데이터를 훈련데이터로 하고 그 이후의 1000을 테스트 데이터로 합니다.

In [168]:
X_train =np.array([i for i in range(1,4001)])
y_train = np.array([isLeap(i) for i in X_train])


In [169]:
X_test = np.array([i for i in range (4001,5001)])
y_test = np.array([isLeap(i) for i in X_test])

In [172]:
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)

X_train_t = torch.tensor(X_train, dtype=torch.float32)
X_test_t  = torch.tensor(X_test, dtype=torch.float32)


y_train_t = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
y_test_t  = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)


#넘파이 배열 > 텐서 배열로 변환
train_dataset = TensorDataset(X_train_t, y_train_t)
test_dataset = TensorDataset(X_test_t, y_test_t)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [174]:
#암환자 모델 고쳐쓰기
class year_leap(nn.Module):
    def __init__(self):
        super(year_leap,self).__init__()
        self.relu = nn.ReLU()
        self.lr1 = nn.Linear(1,3)
        self.lr2 = nn.Linear(3,24) 
        self.lr3 = nn.Linear(24,1) 

    def forward(self,x):
        x = self.lr1(x)
        x = self.relu(x)
        x = self.lr2(x)
        x = self.relu(x)
        x = self.lr3(x)
        return x #대문자 x써서 오류
    

model = year_leap().to(device)


wtf = nn.BCEWithLogitsLoss()  # 손실 함수
opti = optim.Adam(model.parameters(), lr =0.001)

import torch.nn as nn


def train_model(epochs):
    model.train()
    for epoch in range(epochs):
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            opti.zero_grad()
            outputs = model(inputs)

            # outputs: (N,1), labels: (N,1) 조건 필요
            loss = wtf(outputs, labels)
            loss.backward()
            opti.step()

        print(f"Epoch [{epoch+1}/{epochs}] Loss: {loss.item():.4f}")

def evaluate_model():
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for inputs,labels in test_loader:
            inputs,labels = inputs.to(device),labels.to(device)
            outputs = model(inputs)

            predicted = torch.round(torch.sigmoid(outputs))
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
        
        accuarcy = 100 * correct / total
        print(f"T정확도:{accuarcy:.2f}%")

train_model(epochs=100)
evaluate_model()


Epoch [1/100] Loss: 0.5569
Epoch [2/100] Loss: 0.7354
Epoch [3/100] Loss: 0.6496
Epoch [4/100] Loss: 0.5909
Epoch [5/100] Loss: 0.5149
Epoch [6/100] Loss: 0.6912
Epoch [7/100] Loss: 0.5448
Epoch [8/100] Loss: 0.5067
Epoch [9/100] Loss: 0.7530
Epoch [10/100] Loss: 0.5688
Epoch [11/100] Loss: 0.6009
Epoch [12/100] Loss: 0.6235
Epoch [13/100] Loss: 0.5918
Epoch [14/100] Loss: 0.5649
Epoch [15/100] Loss: 0.5284
Epoch [16/100] Loss: 0.4630
Epoch [17/100] Loss: 0.6302
Epoch [18/100] Loss: 0.5813
Epoch [19/100] Loss: 0.6830
Epoch [20/100] Loss: 0.6703
Epoch [21/100] Loss: 0.7307
Epoch [22/100] Loss: 0.4868
Epoch [23/100] Loss: 0.5630
Epoch [24/100] Loss: 0.4924
Epoch [25/100] Loss: 0.4514
Epoch [26/100] Loss: 0.5586
Epoch [27/100] Loss: 0.4941
Epoch [28/100] Loss: 0.7033
Epoch [29/100] Loss: 0.6687
Epoch [30/100] Loss: 0.4940
Epoch [31/100] Loss: 0.4567
Epoch [32/100] Loss: 0.4578
Epoch [33/100] Loss: 0.7050
Epoch [34/100] Loss: 0.5284
Epoch [35/100] Loss: 0.4563
Epoch [36/100] Loss: 0.5269
E